In [1]:
from utilities import *
import networkx as nx

In [85]:
sickChildren = poi("Hospital for Sick Children", "canada")
princessMargaret = poi("princess margaret hospital", "canada")
addictionCenter = poi("center for addiction and mental health", "canada")
torontoWestern = poi("toronto western hospital", "canada")
hillcrest = poi("hillcrest reactivation center", "canada")
runnymede = poi("Runnymede Healthcare Centre", "canada")
salvationArmy = poi("Salvation Army Toronto Grace Health Centre", "canada")
michaelGarron = poi("Michael Garron Hospital", "canada")
hollandBloorview = poi("Holland Bloorview Kids Rehabilitation Hospital", "canada")
northYork = poi("North York General Hospital", "canada")
scarborough = poi("Scarborough Health Network - Birchmount", "canada")


kingStation = poi("King Station", "canada")

POIS = [ 
        # hospitals around UofT -- delivery destination
         sickChildren,
         princessMargaret,
         addictionCenter,
         torontoWestern,
         hillcrest,
         runnymede,
         salvationArmy,
         michaelGarron,
         hollandBloorview,
         northYork,
         scarborough,
       
        # king station -- transportation hub
        kingStation
]

In [86]:
drawPOIS(POIS, zoom=10)

Map(center=[43.689055504166674, -79.38379344383789], close_popup_on_click=False, controls=(ZoomControl(options…

In [87]:
G = nx.DiGraph()

In [88]:
G.add_nodes_from(POIS)

In [89]:
for source in G.nodes():
    for destination in G.nodes():
        if source == destination: continue
        G.add_edge(source, destination, route = source.route_to(destination))

In [127]:
delivery_destinations = [
    {
        'POI':sickChildren, 
        'demand': 50, 
        'time_interval':(0, 300),
        'order': 0,
    },
    {
        'POI':princessMargaret, 
        'demand':60, 
        'time_interval':(300, 500),
        'order': 1,
    },
    {
        'POI':addictionCenter, 
        'demand':40, 
        'time_interval':(500, 700),
        'order': 2,
    },
    {
        'POI':torontoWestern, 
        'demand':85, 
        'time_interval':(700, 900),
        'order': 3,
    },
    {
        'POI':hillcrest,
        'demand':100, 
        'time_interval':(900, 1200),
        'order': 4,
    },
    {
        'POI':runnymede, 
        'demand':120, 
        'time_interval':(1200, 1700),
        'order': 5,
    },
    {
        'POI':salvationArmy, 
        'demand':200, 
        'time_interval':(1700, 2100),
        'order': 6,
    },
    {
        'POI':michaelGarron, 
        'demand':100, 
        'time_interval':(2400, 3100),
        'order': 7,
    },
    {
        'POI':hollandBloorview, 
        'demand': 90, 
        'time_interval':(3100, 4000),
        'order': 8,
    },
    {
        'POI':northYork, 
        'demand':30, 
        'time_interval':(4000, 4500),
        'order': 9,
    },
    {
        'POI':scarborough, 
        'demand':200, 
        'time_interval':(4700, 5200),
        'order': 10,
    },
]

######################################

transportation_hub = {
    'POI': kingStation,
    'demand': -200,
    'time_interval': (-999999, 999999)
}

In [101]:
def evaluate_cost(G, delivery_destinations, transportation_hub):
    cost = 0
    time = 0
    schedule = list()
    
    schedule.append((transportation_hub['POI'], time))
    current_capacity = -1 * transportation_hub['demand']
    complete_tour = [transportation_hub] + delivery_destinations + [transportation_hub]
    
    i = 0
    j = 1
    while j < len(complete_tour):
        start_subroute = complete_tour[i]
        end_subroute = complete_tour[j]
        
        if end_subroute['demand'] > current_capacity:
            complete_tour = complete_tour[:j] + [transportation_hub] + complete_tour[j:]
            continue
            
        current_capacity -= end_subroute['demand']
        time_of_arrival = end_subroute['time_interval']
        
        time += G[start_subroute['POI']][end_subroute['POI']]['route']['duration']
        
        if int(time) not in range(*time_of_arrival):
            cost += (time - min(time_of_arrival))
            if cost < 0 : cost *= -1

        cost += G[start_subroute['POI']][end_subroute['POI']]['route']['length']
        
        schedule.append((end_subroute['POI'], time))
        i += 1
        j += 1   
        
    return cost, schedule
    

In [102]:
evaluate_cost(G, delivery_destinations, transportation_hub)

(166765.5,
 [(Name: King Station ID: 697805124, 0),
  (Name: Hospital for Sick Children ID: 712078929, 247),
  (Name: Princess Margaret Hospital ID: 712078932, 387.8),
  (Name: Centre for Addiction and Mental Health (CAMH) Nicotine Dependence Clinic ID: 328900136,
   522.7),
  (Name: King Station ID: 697805124, 816.9000000000001),
  (Name: Toronto Western Hospital ID: 43805626, 1225.5),
  (Name: Hillcrest Reactivation Centre ID: 7371596851, 1535.6),
  (Name: King Station ID: 697805124, 2165.6),
  (Name: Runnymede Healthcare Centre ID: 486817681, 3120.5),
  (Name: King Station ID: 697805124, 4085.7),
  (Name: Salvation Army Toronto Grace Health Centre ID: 712086304, 4415.3),
  (Name: Michael Garron Hospital ID: 447744987, 5082.2),
  (Name: King Station ID: 697805124, 5868.8),
  (Name: Holland Bloorview Kids Rehabilitation Hospital ID: 684431222, 6858.8),
  (Name: North York General Hospital ID: 684426043, 7683.900000000001),
  (Name: King Station ID: 697805124, 8883.1),
  (Name: Scarbor

In [132]:
schedule = exp_schedule(200, 0.05, 10000)

In [134]:
from tqdm import tqdm
from random import shuffle, randint

In [142]:
%%time
current = delivery_destinations[:]
shuffle(current)

for t in tqdm(range(sys.maxsize)):
    T = schedule(t)
    
    if T < 0.00000001:
        _, solution = evaluate_cost(G, current, transportation_hub)
        break
    
    # generate 5 more random permutations
    neighbors = list()
    for _ in range(5):
        child = current[:]
        i = randint(0, len(child)-1)
        j = randint(0, len(child)-1)
        child[i], child[j] = child[j], child[i]
        neighbors.append(child)
        
    next_choice = random.choice(neighbors)
    
    delta_e = evaluate_cost(G, next_choice, transportation_hub)[0] - evaluate_cost(G, current, transportation_hub)[0]  
    if delta_e < 0 or probability(np.exp(-1 * delta_e/T)):
        current = next_choice
        
    print(evaluate_cost(G, current, transportation_hub)[0])

  0%|          | 475/9223372036854775807 [00:00<979144039051:02:54, 2616.62it/s] 

244233.19999999998
244233.19999999998
239981.40000000002
230414.0
220868.6
220868.6
220868.6
219469.59999999998
219469.59999999998
219469.59999999998
216384.80000000005
210608.50000000006
210608.50000000006
210608.50000000006
207181.30000000002
207181.30000000002
207181.30000000002
207181.30000000002
207181.30000000002
207181.30000000002
207181.30000000002
204112.30000000002
204112.30000000002
193648.19999999998
193648.19999999998
193648.19999999998
186868.9
186868.9
186868.9
185817.1
185817.1
185817.1
185147.90000000002
185147.90000000002
185147.90000000002
185147.90000000002
185147.90000000002
185147.90000000002
185147.90000000002
185147.90000000002
185147.90000000002
175849.1
175849.1
175849.1
175849.1
175849.1
175849.1
175849.1
164740.3
164740.3
162937.1
162937.1
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026.8
146026

In [143]:
solution

[(Name: King Station ID: 697805124, 0),
 (Name: Hospital for Sick Children ID: 712078929, 247),
 (Name: Centre for Addiction and Mental Health (CAMH) Nicotine Dependence Clinic ID: 328900136,
  405.1),
 (Name: Princess Margaret Hospital ID: 712078932, 503.5),
 (Name: King Station ID: 697805124, 755.6),
 (Name: Salvation Army Toronto Grace Health Centre ID: 712086304, 1085.2),
 (Name: King Station ID: 697805124, 1399.9),
 (Name: Runnymede Healthcare Centre ID: 486817681, 2354.8),
 (Name: Hillcrest Reactivation Centre ID: 7371596851, 2915.0),
 (Name: King Station ID: 697805124, 3545.0),
 (Name: Michael Garron Hospital ID: 447744987, 4337.6),
 (Name: Holland Bloorview Kids Rehabilitation Hospital ID: 684431222, 5040.8),
 (Name: King Station ID: 697805124, 6011.400000000001),
 (Name: Toronto Western Hospital ID: 43805626, 6420.000000000001),
 (Name: King Station ID: 697805124, 6839.500000000001),
 (Name: Scarborough Health Network - Birchmount Hospital ID: 681295730,
  8281.0),
 (Name: Nor

In [159]:
from ipywidgets import HTML
from ipyleaflet import Map, Marker, AntPath, Popup, CircleMarker, Polyline

In [168]:
def drawTour(solution, zoom = 11):
    centerLat, centerLog = 0, 0

    # taking the average of all latitude and longitude of all the POIs
    # to get the center of the map 
    POIs = [pair[0] for pair in solution]
    for poi in POIs:
        centerLat += poi.coordinates[0]
        centerLog += poi.coordinates[1]
    centerLat /= len(POIs)
    centerLog /= len(POIs)
    center = (centerLog, centerLat)
    
    m = Map(center=center, zoom=zoom, close_popup_on_click=False)
    
    for poi in POIS:
        name = poi.address.split(",")[0]
        marker = Marker(location=poi.coordinates[::-1])
        text = HTML()
        text.value = f"{name}"
        marker.popup = text
        m.add_layer(marker)
        
    for start, end in zip(POIs, POIs[1:]):
        route = G[start][end]['route']['coords']
        for u, v in zip(route[0:], route[1:]):
            step = map(list, [u, v])
            step_path = Polyline(
                locations = [*step],
                color='black',
            )
            m.add_layer(step_path)

    source = POIs[0]
    src_marker = CircleMarker(location = source.coordinates[::-1], radius = 10, color = "red")
    m.add_layer(src_marker)
    
    return m

In [169]:
drawTour(solution)

Map(center=[43.67554456944444, -79.3817865625586], close_popup_on_click=False, controls=(ZoomControl(options=[…